In [ ]:
import pandas as pd
import psycopg2
import import_ipynb
from Utils import *
import os
import time
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()

connection_params = {
    'dbname': os.getenv("DB_NAME"),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'host': os.getenv("DB_HOST"),
    'port': os.getenv("DB_PORT")
}

print("Parámetros de conexión cargados.")

In [ ]:
# Función para normalizar texto
def normalize_text(series):
    return series.astype(str).str.lower().str.strip().str.replace(r'[.,;:!?()\\[\\]{}¿¡-]', '', regex=True)

# Conectar a la base de datos
conn = psycopg2.connect(**connection_params)
cur = conn.cursor()
start_time = time.time()

In [ ]:
# Cargar datos a DataFrames
try:
    df_paciente = pd.read_sql("SELECT * FROM paciente", conn)
    df_infolab = pd.read_sql("SELECT * FROM infolab", conn)
    #df_paciente_clean = pd.read_sql("SELECT * FROM paciente_clean", conn)
    df_infolab_clean = pd.read_sql("SELECT * FROM infolab_clean", conn)
    print("Datos cargados exitosamente.")
except Exception as e:
    print(f"Error al cargar datos: {e}")
    raise

In [ ]:
# Transferir pacientes únicos a paciente_clean (con actualización si existe)
'''
try:
    
    cur.execute("""
        SELECT column_name FROM information_schema.columns 
        WHERE table_name = 'paciente_clean';
    """)
    cols = [row[0] for row in cur.fetchall()]
    
    df_paciente_unique = df_paciente.drop_duplicates(subset=['id_paciente'])
    if not df_paciente_unique.empty:
        df_to_insert = df_paciente_unique[cols]
        data = [tuple(row) for row in df_to_insert.itertuples(index=False)]
        
        placeholders = ','.join(['%s'] * len(cols))
        cols_str = ','.join(cols)
        excluded_str = ','.join(f"{col} = EXCLUDED.{col}" for col in cols if col != 'id_paciente')
        
        insert_query = f"""
            INSERT INTO paciente_clean ({cols_str})
            VALUES ({placeholders})
            ON CONFLICT (id_paciente) DO UPDATE SET {excluded_str};
        """
        
        cur.executemany(insert_query, data)
        conn.commit()
        print(f"✓ Pacientes insertados/actualizados: {len(data)}")
    else:
        print("No hay pacientes nuevos para insertar.")
except Exception as e:
    print(f"Error: {e}")
    conn.rollback()
    raise
    '''

In [ ]:
# Normalizar columnas clave para infolab
cols_infolab = ['servicio', 'muestra_remitida', 'descripcion_macroscopica', 'descripcion_microscopica', 'diagnostico','fecha_toma_muestra','entidad','medico_ordenante']
for col in cols_infolab:
    # Reemplazar NULL/NaN con string vacío antes de normalizar
    df_infolab[f"{col}_norm"] = normalize_text(df_infolab[col].fillna(''))
    df_infolab_clean[f"{col}_norm"] = normalize_text(df_infolab_clean[col].fillna(''))
print("Normalización completada.")

In [ ]:
#se trae paciente_clean actualizado
#df_paciente_clean = pd.read_sql("SELECT * FROM paciente_clean", conn)

# Filtrar infolab nuevos: relacionados con pacientes existentes y no duplicados
df_infolab_dedup = df_infolab.drop_duplicates(subset=['id_paciente'] + [f"{c}_norm" for c in cols_infolab])

# Filtrar relacionados con pacientes existentes
df_infolab_relacionados = df_infolab_dedup[
    df_infolab_dedup['id_paciente'].isin(df_paciente['id_paciente'])
].copy()

# Crear clave compuesta (con fillna para consistencia)
df_infolab_relacionados['_key'] = (
    df_infolab_relacionados['id_paciente'].fillna('').astype(str) + '|' +
    df_infolab_relacionados['servicio_norm'].fillna('').astype(str) + '|' +
    df_infolab_relacionados['muestra_remitida_norm'].fillna('').astype(str) + '|' +
    df_infolab_relacionados['descripcion_macroscopica_norm'].fillna('').astype(str) + '|' +
    df_infolab_relacionados['descripcion_microscopica_norm'].fillna('').astype(str) + '|' +
    df_infolab_relacionados['diagnostico_norm'].fillna('').astype(str) + '|' +
    df_infolab_relacionados['fecha_toma_muestra_norm'].fillna('').astype(str) + '|' +
    df_infolab_relacionados['entidad_norm'].fillna('').astype(str) + '|' +
    df_infolab_relacionados['medico_ordenante_norm'].fillna('').astype(str)
)

if not df_infolab_clean.empty:
    keys_existentes = set(
        df_infolab_clean['id_paciente'].fillna('').astype(str) + '|' +
        df_infolab_clean['servicio_norm'].fillna('').astype(str) + '|' +
        df_infolab_clean['muestra_remitida_norm'].fillna('').astype(str) + '|' +
        df_infolab_clean['descripcion_macroscopica_norm'].fillna('').astype(str) + '|' +
        df_infolab_clean['descripcion_microscopica_norm'].fillna('').astype(str) + '|' +
        df_infolab_clean['diagnostico_norm'].fillna('').astype(str) + '|' +
        df_infolab_clean['fecha_toma_muestra_norm'].fillna('').astype(str) + '|' +
        df_infolab_clean['entidad_norm'].fillna('').astype(str) + '|' +
        df_infolab_clean['medico_ordenante_norm'].fillna('').astype(str)
    )
    print(f"Keys existentes en infolab_clean: {len(keys_existentes)}")
else:
    keys_existentes = set()
    print("infolab_clean está vacío, todos los registros relacionados serán insertados.")

# Filtrar registros nuevos
df_infolab_filtered = df_infolab_relacionados[
    ~df_infolab_relacionados['_key'].isin(keys_existentes)
].drop(columns=['_key'] + [f"{c}_norm" for c in cols_infolab])

print(f"Registros filtrados para infolab_clean: {len(df_infolab_filtered)}")

In [ ]:
try:
    if not df_infolab_filtered.empty:
        # Definir las columnas a insertar en el orden correcto
        cols_to_insert = [
            'fecha_toma_muestra', 'fecha_ingreso', 'fecha_informe', 'entidad', 'eps', 
            'servicio', 'muestra_remitida', 'descripcion_macroscopica', 
            'descripcion_microscopica', 'diagnostico', 'comentario', 'id_paciente', 
            'archivo', 'fuente', 'cie10', 'edad', 'sexo', 'diagnostico_clinico', 
            'observacion', 'medico_ordenante', 'medico_remitente'
        ]
        
        # Seleccionar solo las columnas necesarias
        df_to_insert = df_infolab_filtered[cols_to_insert]
        
        # Convertir a lista de tuplas
        data = [tuple(row) for row in df_to_insert.itertuples(index=False)]
        
        # Insertar usando executemany (más eficiente)
        cur.executemany("""
            INSERT INTO infolab_clean (fecha_toma_muestra, fecha_ingreso, fecha_informe, entidad, eps, servicio, muestra_remitida, descripcion_macroscopica, descripcion_microscopica, diagnostico, comentario, id_paciente, archivo, fuente, cie10, edad, sexo, diagnostico_clinico, observacion, medico_ordenante, medico_remitente)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, data)
        
        conn.commit()
        print(f"✓ Registros insertados en infolab_clean: {len(df_infolab_filtered)}")
    else:
        print("No hay registros nuevos para insertar en infolab_clean")
        
except Exception as e:
    conn.rollback()
    print(f"✗ Error al insertar infolab: {e}")
    raise

In [ ]:
# Confirmar cambios y cerrar conexión
try:
    conn.commit()
    end_time = time.time()
    print(f"Transferencia completada exitosamente en {end_time - start_time:.2f} segundos.")    
except Exception as e:
    print(f"Error al confirmar cambios: {e}")
    conn.rollback()
    raise
finally:
    cur.close()
    conn.close()
    print("Conexión cerrada.")